In [ ]:
##Final script used to generate mtl extracted betas

In [ ]:
import os
import numpy as np
import nibabel as nib
import h5py
import gc

In [ ]:
SAVE_BASE_PATH = "/home/jovyan/cache/memoryNSD/" #change this based on where you want to save the h5 files

In [ ]:
# === Step 1: Download function ===
def download_nsd_data(subject, session, save_path=SAVE_BASE_PATH):
    subj_str = f"subj{subject:02d}"
    session_str = f"session{session:02d}"

    url = f"https://natural-scenes-dataset.s3.amazonaws.com/nsddata_betas/ppdata/{subj_str}/func1mm/betas_fithrf_GLMdenoise_RR/betas_{session_str}.nii.gz"
    file_path = os.path.join(save_path, f"{subj_str}_betas_{session_str}.nii.gz")

    os.system(f"wget -q -O {file_path} {url}")
    print(f"Downloaded beta file: {file_path}")
    return file_path

def download_mtl_mask(subject, save_path=SAVE_BASE_PATH):
    subj_str = f"subj{subject:02d}"
    url = f"https://natural-scenes-dataset.s3.amazonaws.com/nsddata/ppdata/{subj_str}/func1mm/roi/MTL.nii.gz"
    file_path = os.path.join(save_path, f"{subj_str}_MTL.nii.gz")

    os.system(f"wget -q -O {file_path} {url}")
    print(f"Downloaded MTL mask: {file_path}")
    return file_path

In [ ]:
# === Step 2: Load NIfTI ===
def load_betas(file_path):
    img = nib.load(file_path)
    return img.get_fdata(dtype=np.float32) / 300.0

def load_mask(file_path):
    img = nib.load(file_path)
    return img.get_fdata(dtype=np.float32)

In [ ]:
# === Step 3: Extract ROI betas ===
def extract_roi_betas(fmri_data, roi_indices):
    roi_betas = []
    for i in range(len(roi_indices[0])):
        x, y, z = roi_indices[0][i], roi_indices[1][i], roi_indices[2][i]
        voxel_betas = fmri_data[x, y, z]
        voxel_betas_normalized = (voxel_betas - np.mean(voxel_betas)) / np.std(voxel_betas)
        roi_betas.append(voxel_betas_normalized)
    return np.array(roi_betas).T

def extract_mtl_rois(fmri_data, mtl_data):
    mtl_rois = {}
    for roi_label in range(1, 11):  # Labels 1 to 10
        roi_indices = np.where(mtl_data == roi_label)
        if len(roi_indices[0]) > 0:
            roi_betas = extract_roi_betas(fmri_data, roi_indices)
            mtl_rois[roi_label] = roi_betas
    return mtl_rois

In [ ]:
# === Step 4: Save HDF5 ===
def save_hdf5(data, file_path):
    with h5py.File(file_path, 'w') as hf:
        for session, rois in data.items():
            session_group = hf.create_group(f"session{session:02d}")
            for roi_label, roi_data in rois.items():
                session_group.create_dataset(f"roi_{roi_label}", data=roi_data)
    print(f"Saved all_mtl_sessions to {file_path}")

In [ ]:
# === Step 5: Process all subjects ===
for subject in range(1, 9):  # Change range for more subjects (e.g., range(1, 9))
    print(f"\n--- Processing Subject {subject:02d} ---")
    all_mtl_sessions = {}

    # Download subject-specific MTL mask
    mtl_mask_path = download_mtl_mask(subject)
    MTL_data = load_mask(mtl_mask_path)

    for session in range(1, 41):  # Sessions 1 to 40
        print(f"Processing Session {session} for Subject {subject:02d}...")
        beta_path = download_nsd_data(subject=subject, session=session)

        try:
            betas = load_betas(beta_path)
            all_mtl_sessions[session] = extract_mtl_rois(betas, MTL_data)
            del betas
            gc.collect()
            os.remove(beta_path)
        except Exception as e:
            print(f"Error processing subj{subject:02d} session{session:02d}: {e}")
            # Clean up broken or empty beta file
            if os.path.exists(beta_path):
                os.remove(beta_path)

    save_path = os.path.join(SAVE_BASE_PATH, f"subj{subject:02d}_all_mtl_sessions.h5")

    save_hdf5(all_mtl_sessions, save_path)
    os.remove(mtl_mask_path)  # Clean up MTL mask after use